# Подготовка данных

### Ч/Б + адаптивная бинаризация

In [ ]:
!pip install opencv-python

In [ ]:
!pip install tqdm

In [4]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def convert_to_bw(input_dir, output_dir, threshold=180):
    """
    Конвертирует все изображения в папке в чёрно-белые с адаптивной бинаризацией
    :param input_dir: Папка с исходными изображениями
    :param output_dir: Папка для результатов
    :param threshold: Порог бинаризации (0-255)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for root, _, files in os.walk(input_dir):
        for file in tqdm(files, desc="Обработка изображений"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')):
                rel_path = os.path.relpath(root, input_dir)
                os.makedirs(os.path.join(output_dir, rel_path), exist_ok=True)
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # Адаптивная бинаризация
                bw = cv2.adaptiveThreshold(
                    gray, 255,
                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                    cv2.THRESH_BINARY_INV,
                    blockSize=21,
                    C=7
                )
                
                output_path = os.path.join(output_dir, rel_path, file)
                cv2.imwrite(output_path, bw)

input_folder = "data/dataset/raw/"  # Папка с исходными изображениями
output_folder = "data/dataset/processed/"  # Папка для чёрно-белых версий

convert_to_bw(input_folder, output_folder)

Обработка изображений: 0it [00:00, ?it/s]
Обработка изображений: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.58it/s]


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Загрузка размеченных данных
def load_data(annotations_path, img_dir):
    images = []
    labels = []
    class_map = {'а':0, 'б':1, ... 'я':32}  # Все 33 буквы
    
    with open(annotations_path) as f:
        data = json.load(f)
    
    for item in data:
        img = cv2.imread(f"{img_dir}/{item['data']['image']}", cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))  # Нормализация размера
        images.append(img)
        
        label = item['annotations'][0]['result'][0]['value']['rectanglelabels'][0]
        labels.append(class_map[label])
    
    return np.array(images), np.array(labels)

# Использование
X, y = load_data("export.json", "dataset/images")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
import cv2
import numpy as np
import json
import os
from sklearn.model_selection import train_test_split

def load_data(annotations_path, processed_dir):
    # Русский алфавит с нумерацией
    russian_alphabet = ['ru_1', 'ru_2', 'ru_3', 'ru_4']
    
    class_map = {idx+1: letter for idx, letter in enumerate(russian_alphabet)}
    letter_to_num = {letter: num for num, letter in class_map.items()}
    
    images = []
    labels = []
    
    with open(annotations_path) as f:
        annotations = json.load(f)
    
    # Обработка каждого элемента аннотаций
    for item in annotations:
        try:
            # Получаем метку класса
            label = None
            for result in item['annotations'][0]['result']:
                if 'labels' in result['value']:
                    label = result['value']['labels'][0]
                    break
                elif 'text' in result['value']:
                    label = result['value']['text'][0]
                    break
            
            if not label or label not in letter_to_num:
                print(f"Пропущен элемент с некорректной меткой: {label}")
                continue
                
            class_num = letter_to_num[label]
            
            # Формируем путь к изображению
            img_dir = os.path.join(processed_dir, f"ru_{class_num}")
            if not os.path.exists(img_dir):
                print(f"Папка не найдена: {img_dir}")
                continue
                
            # Ищем любой jpg файл в папке
            img_files = [f for f in os.listdir(img_dir) if f.lower().endswith('.jpg')]
            if not img_files:
                print(f"Нет jpg файлов в папке: {img_dir}")
                continue
                
            img_path = os.path.join(img_dir, img_files[0])
            
            # Загрузка и обработка изображения
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Не удалось загрузить: {img_path}")
                continue
                
            img = cv2.resize(img, (3000, 4000))
            img = img.astype(np.float32) / 255.0
            
            images.append(img)
            labels.append(class_num - 1)  # Классы 0-32
            
        except Exception as e:
            print(f"Ошибка обработки элемента: {str(e)}")
            continue
    
    # Проверка и преобразование данных
    if not images:
        raise ValueError("Не загружено ни одного изображения!")
        
    X = np.array(images)
    y = np.array(labels)
    X = np.expand_dims(X, axis=1)  # Добавляем размерность канала
    
    return X, y, class_map

# Использование
try:
    annotations_path = "data/dataset/export.json"
    processed_dir = "data/dataset/processed/"
    
    X, y, class_map = load_data(annotations_path, processed_dir)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42,
        stratify=y
    )
    
    print(f"Успешно загружено {len(X)} изображений")
    print(f"Размерности: X_train {X_train.shape}, X_test {X_test.shape}")
    print("Соответствие классов:", class_map)
    
except Exception as e:
    print(f"Ошибка: {str(e)}")

Ошибка: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.


In [25]:
with open("dataset/export.json") as f:
    data = json.load(f)
print(json.dumps(data[0], indent=2, ensure_ascii=False))  # Первый элемент

{
  "id": 1,
  "annotations": [
    {
      "id": 2,
      "completed_by": 1,
      "result": [
        {
          "original_width": 3000,
          "original_height": 4000,
          "image_rotation": 0,
          "value": {
            "x": 11.991325884916344,
            "y": 2.3026821445672696,
            "width": 3.0123137488678715,
            "height": 1.9551074812363602,
            "rotation": 0
          },
          "id": "ZoYa6MKP9d",
          "from_name": "bbox",
          "to_name": "image",
          "type": "rectangle",
          "origin": "manual"
        },
        {
          "original_width": 3000,
          "original_height": 4000,
          "image_rotation": 0,
          "value": {
            "x": 11.991325884916344,
            "y": 2.3026821445672696,
            "width": 3.0123137488678715,
            "height": 1.9551074812363602,
            "rotation": 0,
            "text": [
              "Р°"
            ]
          },
          "id": "ZoYa6MKP9d",
  

In [21]:
import cv2
import numpy as np
import json
import os
from sklearn.model_selection import train_test_split

def load_data(annotations_path, processed_dir):
    # Русский алфавит с нумерацией
    russian_alphabet = [
        'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й',
        'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф',
        'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'
    ]
    class_map = {idx+1: letter for idx, letter in enumerate(russian_alphabet)}
    letter_to_num = {letter: num for num, letter in class_map.items()}
    
    images = []
    labels = []
    
    # Загрузка аннотаций
    with open(annotations_path) as f:
        annotations = json.load(f)
    
    # Обработка каждого элемента аннотаций
    for item in annotations:
        try:
            # Получаем метку класса
            label = None
            for result in item['annotations'][0]['result']:
                if 'labels' in result['value']:
                    label = result['value']['labels'][0]
                    break
                elif 'text' in result['value']:
                    label = result['value']['text'][0]
                    break
            
            if not label or label not in letter_to_num:
                print(f"Пропущен элемент с некорректной меткой: {label}")
                continue
                
            class_num = letter_to_num[label]
            
            # Формируем путь к изображению
            img_dir = os.path.join(processed_dir, f"ru_{class_num}")
            if not os.path.exists(img_dir):
                print(f"Папка не найдена: {img_dir}")
                continue
                
            # Ищем любой jpg файл в папке
            img_files = [f for f in os.listdir(img_dir) if f.lower().endswith('.jpg')]
            if not img_files:
                print(f"Нет jpg файлов в папке: {img_dir}")
                continue
                
            img_path = os.path.join(img_dir, img_files[0])
            
            # Загрузка и обработка изображения
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Не удалось загрузить: {img_path}")
                continue
                
            img = cv2.resize(img, (64, 64))
            img = img.astype(np.float32) / 255.0
            
            images.append(img)
            labels.append(class_num - 1)  # Классы 0-32
            
        except Exception as e:
            print(f"Ошибка обработки элемента: {str(e)}")
            continue
    
    # Проверка и преобразование данных
    if not images:
        raise ValueError("Не загружено ни одного изображения!")
        
    X = np.array(images)
    y = np.array(labels)
    X = np.expand_dims(X, axis=1)  # Добавляем размерность канала
    
    return X, y, class_map

# Использование
try:
    annotations_path = "dataset/export.json"
    processed_dir = "dataset/processed"
    
    X, y, class_map = load_data(annotations_path, processed_dir)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42,
        stratify=y
    )
    
    print(f"Успешно загружено {len(X)} изображений")
    print(f"Размерности: X_train {X_train.shape}, X_test {X_test.shape}")
    print("Соответствие классов:", class_map)
    
except Exception as e:
    print(f"Ошибка: {str(e)}")

Пропущен элемент с некорректной меткой: Р°
Пропущен элемент с некорректной меткой: Р±
Пропущен элемент с некорректной меткой: РІ
Пропущен элемент с некорректной меткой: Рі
Пропущен элемент с некорректной меткой: Рґ
Пропущен элемент с некорректной меткой: Рµ
Пропущен элемент с некорректной меткой: С‘
Пропущен элемент с некорректной меткой: Р¶
Пропущен элемент с некорректной меткой: Р·
Пропущен элемент с некорректной меткой: Рё
Пропущен элемент с некорректной меткой: Р№
Пропущен элемент с некорректной меткой: Рє
Пропущен элемент с некорректной меткой: Р»
Пропущен элемент с некорректной меткой: Рј
Пропущен элемент с некорректной меткой: РЅ
Пропущен элемент с некорректной меткой: Рѕ
Пропущен элемент с некорректной меткой: Рї
Пропущен элемент с некорректной меткой: СЂ
Пропущен элемент с некорректной меткой: СЃ
Пропущен элемент с некорректной меткой: С‚
Пропущен элемент с некорректной меткой: Сѓ
Пропущен элемент с некорректной меткой: С„
Пропущен элемент с некорректной меткой: С…
Пропущен эл

# Архитектура нейросети

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class HandwritingCNN(nn.Module):
    def __init__(self, num_classes=33):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*16*16, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = HandwritingCNN()

# Обучение модели

In [ ]:
# Преобразование данных в тензоры
X_train_t = torch.FloatTensor(X_train).unsqueeze(1)  # Добавляем размерность канала
y_train_t = torch.LongTensor(y_train)

# Loss и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Цикл обучения
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Проверка точности

In [ ]:
with torch.no_grad():
    X_test_t = torch.FloatTensor(X_test).unsqueeze(1)
    outputs = model(X_test_t)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == torch.LongTensor(y_test)).float().mean()
    print(f"Test Accuracy: {accuracy:.4f}")

# Сохранение и загрузка модели

In [ ]:
# Сохранение
torch.save(model.state_dict(), "handwriting_cnn.pt")

# Загрузка
model = HandwritingCNN()
model.load_state_dict(torch.load("handwriting_cnn.pt"))
model.eval()

# Инференс (предсказание для нового изображения)

In [ ]:
def predict_letter(model, img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img_tensor = torch.FloatTensor(img).unsqueeze(0).unsqueeze(0)
    
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
    
    idx_to_char = {v:k for k,v in class_map.items()}
    return idx_to_char[predicted.item()]

# Использование
print(predict_letter(model, "test_letter.jpg"))